# GWAS in the cloud
## Overview
We adapted the NIH CFDE tutorial from [here](https://training.nih-cfde.org/en/latest/Bioinformatic-Analyses/GWAS-in-the-cloud/background/) and fit it to a notebook. We have greatly simplified the instructions, so if you need or want more details, look at the full tutorial to find out more.

Most of this notebook is written in Bash, but expects that you are using a Python kernel, until step 3, plotting where you will need to switch your kernel to R.

## Learning Objectives
The goal is to learn how to execute a GWAS analysis in a cloud environment

## Prerequisites
+ You only need access to a Sagemaker notebook environment to run this notebook

## Get Started

### Install packages and set up environment

#### Download the data
use %%bash to denote a bash block. You can also use '!' to denote a single bash command within a Python notebook

In [ ]:
! mkdir GWAS

In [ ]:
%%bash
curl -LO https://de.cyverse.org/dl/d/E0A502CC-F806-4857-9C3A-BAEAA0CCC694/pruned_coatColor_maf_geno.vcf.gz
curl -LO https://de.cyverse.org/dl/d/3B5C1853-C092-488C-8C2F-CE6E8526E96B/coatColor.pheno

In [ ]:
%%bash
mv *.gz GWAS
mv *.pheno GWAS
ls GWAS

### Install dependencies

In [ ]:
# install mamba
! curl -L -O https://github.com/conda-forge/miniforge/releases/latest/download/Mambaforge-$(uname)-$(uname -m).sh
! bash Mambaforge-$(uname)-$(uname -m).sh -b -p $HOME/mambaforge

In [ ]:
# add to your path
import os
os.environ["PATH"] += os.pathsep + os.environ["HOME"]+"/mambaforge/bin"

In [ ]:
# install everything else
! mamba install -y -c bioconda plink vcftools

## Analyze

### Make map and ped files from the vcf file to feed into plink

In [ ]:
%%bash

cd GWAS
pwd

In [ ]:
! vcftools --gzvcf pruned_coatColor_maf_geno.vcf.gz --plink --out coatColor

### Create a list of minor alleles
For more info on these terms, look at step 2 at https://training.nih-cfde.org/en/latest/Bioinformatic-Analyses/GWAS-in-the-cloud/analyze/

In [ ]:
#unzip vcf
! vcftools --gzvcf pruned_coatColor_maf_geno.vcf.gz --recode --out pruned_coatColor_maf_geno

In [ ]:
#create list of minor alleles
! cat pruned_coatColor_maf_geno.recode.vcf | awk 'BEGIN{FS="\t";OFS="\t";}/#/{next;}{{if($3==".")$3=$1":"$2;}print $3,$5;}'  > minor_alleles

In [ ]:
! head minor_alleles

### Run quality controls

In [ ]:
#calculate missingness per locus
! plink --file coatColor --make-pheno coatColor.pheno "yellow" --missing --out miss_stat --noweb --dog --reference-allele minor_alleles --allow-no-sex --adjust

In [ ]:
#take a look at lmiss, which is the per locus rates of missingness
! head miss_stat.lmiss

In [ ]:
#peek at imiss which is the individual rates of missingness
! head miss_stat.imiss

### Convert to plink binary format

In [ ]:
! plink --file coatColor --allow-no-sex --dog --make-bed --noweb --out coatColor.binary

### Run a simple association step (the GWAS part!)

In [ ]:
! plink --bfile coatColor.binary --make-pheno coatColor.pheno "yellow" --assoc --reference-allele minor_alleles --allow-no-sex --adjust --dog --noweb --out coatColor

### Identify statistical cutoffs
This code finds the equivalent of 0.05 and 0.01 p value in the negative-log-transformed p values file. We will use these cutoffs to draw horizontal lines in the Manhattan plot for visualization of haplotypes that cross the 0.05 and 0.01 statistical threshold (i.e. have a statistically significant association with yellow coat color)

In [ ]:
%%bash
unad_cutoff_sug=$(tail -n+2 coatColor.assoc.adjusted | awk '$10>=0.05' | head -n1 | awk '{print $3}')
unad_cutoff_conf=$(tail -n+2 coatColor.assoc.adjusted | awk '$10>=0.01' | head -n1 | awk '{print $3}')

## Plotting
In this tutorial, plotting is done in R, so at this point you can change your kernel to R in the top right. Wait for it to say 'idle' in the bottom left, then continue. You could also plot using Python native packages and maintain the Python notebook kernel.

### Install qqman

In [ ]:
install.packages('qqman', contriburl=contrib.url('http://cran.r-project.org/'))

### Run the plotting function

In [ ]:
#make sure you are still CD in GWAS, when you change kernel it may reset to home
setwd('GWAS')

In [ ]:
require(qqman)

In [ ]:
data=read.table("coatColor.assoc", header=TRUE)

In [ ]:
data=data[!is.na(data$P),]

In [ ]:
manhattan(data, p = "P", col = c("blue4", "orange3"),
          suggestiveline = 12,
          genomewideline = 15,
          chrlabs = c(1:38, "X"), annotateTop=TRUE, cex = 1.2)

In our graph, haplotypes in four parts of the genome (chromosome 2, 5, 28 and X) are found to be associated with an increased occurrence of the yellow coat color phenotype.

The top associated mutation is a nonsense SNP in the gene MC1R known to control pigment production. The MC1R allele encoding yellow coat color contains a single base change (from C to T) at the 916th nucleotide.

### Conclusion
Here we learned how to run a simple GWAS analysis in the cloud

## Clean up
Make sure you shut down this VM, or delete it if you don't plan to use if further.

You can also [delete the buckets](https://docs.aws.amazon.com/AmazonS3/latest/userguide/delete-bucket.html) if you don't want to pay for the data: `aws s3 rb s3://bucket-name --force`